In [1]:
from ctypes import *
from time import sleep

In [2]:
# Custom Exceptions
class Error(Exception):
    """Base class for exceptions in this module.
    :param msg: Error message associated with the exception
    :type msg: str
    :ivar msg: Error message associated with the exception
    """

    def __init__(self, msg):
        self.msg = msg

    def __str__(self):
        return self.msg


class DLLError(Error):
    """Exception raised opening dgilib.dll. dgilib.dll could not be found in the specicied path"""

    pass


class DeviceIndexError(Error):
    """Exception raised selecting device: the device_index exeeds the device_count"""

    pass


class DeviceReturnError(Error):
    """Exception raised: DGILib returned non-zero value"""

    pass


class DeviceConnectionError(Error):
    """Exception raised: Could not connect to the device"""

    pass


class DeviceArgumentError(Error):
    """Exception raised parsing arguments"""

    pass



In [3]:
GET_STRING_SIZE = 100
NUM_INTERFACES = 10
NUM_CONFIG_IDS = 255
NUM_CALIBRATION = 255
BUFFER_SIZE = 10000000

INTERFACE_TIMESTAMP  = 0x00 #   0 Service interface which appends timestamps to all received events on associated interfaces.
INTERFACE_SPI        = 0x20 #  32 Communicates directly over SPI in Slave mode.
INTERFACE_USART      = 0x21 #  33 Communicates directly over USART in Slave mode.
INTERFACE_I2C        = 0x22 #  34 Communicates directly over I2C in Slave mode.
INTERFACE_GPIO       = 0x30 #  48 Monitors and controls the state of GPIO pins.
INTERFACE_POWER_DATA = 0x40 #  64 Receives data from the attached power measurement co-processors.
INTERFACE_POWER_SYNC = 0x41 #  65 Receives sync events from the attached power measurement co-processors.
INTERFACE_RESERVED   = 0xFF # 255 Special identifier used to indicate no interface.

# Circuit types
OLD_XAM = 0x00
XAM     = 0x10
PAM     = 0x11
UNKNOWN = 0xFF

In [4]:
# https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER

"""This module provides Python bindings for the Discovery API of DGILib."""

__license__ = "MIT"
__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "1.6.0"
__revision__ = " $Id: dgilib_discovery.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"


class DGILibDiscovery(object):
    """Python bindings for DGILib Discovery.
    
    DGILib is a Dynamic-Link Library (DLL) to help software applications communicate with Data Gateway
    Interface (DGI) devices. See the Data Gateway Interface user guide for further details. DGILib handles
    the low-level USB communication and adds a level of buffering for minimizing the chance of overflows.
    """

    def __enter__(self):
        """
        :raises: :exc:`DeviceIndexError`
        """

        self.discover()
        device_count = self.get_device_count()

        if self.device_sn is None:
            if self.device_index is None:
                self.device_index = 0
            elif self.device_index > device_count - 1:
                raise DeviceIndexError(
                    f"Discovered {device_count} devices so could not select device with index {self.device_index}."
                )
            self.device_sn = self.get_device_serial(self.device_index)

        # UNTESTED:
        # if msd_mode:
        #     res = self.set_mode(sn, 1)
        #     print(f"\t{res} set_mode 1")

        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if self.verbose:
            print("bye from Discovery")

        """
TODO?
2.1.1. initialize_status_change_notification
Initializes the system necessary for using the status change notification callback mechanisms. A handle
will be created to keep track of the registered callbacks. This function must always be called before
registering and unregistering notification callbacks.
Function definition
void initialize_status_change_notification(uint32_t* handlep)
Parameters
handlep Pointer to a variable that will hold the handle
2.1.2. uninitialize_status_change_notification
Uninitializes the status change notification callback mechanisms. This function must be called when
shutting down to clean up memory allocations.
Function definition
void uninitialize_status_change_notification(uint32_t handle)
Parameters
handle Handle to uninitialize
2.1.3. register_for_device_status_change_notifications
Registers provided function pointer with the device status change mechanism. Whenever there is a
change (device connected or disconnected) the callback will be executed. Note that it is not allowed to
connect to a device in the context of the callback function. The callback function has the following
definition: typedef void (*DeviceStatusChangedCallBack)(char* device_name, char* device_serial, BOOL
connected)
Function definition
void register_for_device_status_change_notifications(uint32_t handle, DeviceStatusChangedCallBack
deviceStatusChangedCallBack)
Parameters
handle Handle to change notification mechanisms
deviceStatusChangedCallBack Function pointer that will be called when the devices change
2.1.4. unregister_for_device_status_change_notifications
Unregisters previously registered function pointer from the device status change mechanism.
Function definition
void unregister_for_device_status_change_notifications(uint32_t handle, DeviceStatusChangedCallBack
deviceStatusChangedCallBack)
Parameters
handle Handle to change notification mechanisms
deviceStatusChangedCallBack Function pointer that will be removed
        """

    def discover(self):
        """`discover`

        Triggers a scan to find available devices in the system. The result will be immediately available through
        the `get_device_count`, `get_device_name` and `get_device_serial` functions.

        `void discover(void)`
        """

        self.dgilib.discover()

    def get_device_count(self):
        """`get_device_count`

        Returns the number of devices detected.

        `int get_device_count(void)`
        
        :return: The number of devices detected
        :rtype: int
        """

        device_count = self.dgilib.get_device_count()
        if self.verbose:
            print(f"device_count: {device_count}")
        return device_count

    def get_device_name(self, index):
        """`get_device_name`

        Gets the name of a detected device.

        `int get_device_name(int index, char* name)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *index* | Index of device ranges from 0 to `get_device_count` - 1 |
        | *name* | Pointer to buffer where name of device can be stored. 100 or more bytes must be allocated |
        +------------+------------+
        
        :param index: Index of device ranges from 0 to `get_device_count` - 1
        :type index: int
        :return: The name of a detected device
        :rtype: str
        :raises: :exc:`DeviceReturnError`
        """

        name = create_string_buffer(GET_STRING_SIZE)
        res = self.dgilib.get_device_name(index, byref(name))
        if self.verbose:
            print(f"\t{res} get_device_name: {name.value}")
        if res:
            raise DeviceReturnError(f"get_device_name returned: {res}")
        return name.value

    def get_device_serial(self, index):
        """`get_device_serial`

        Gets the serial number of a detected device.

        `int get_device_serial(int index, char* sn)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *index* | Index of device ranges from 0 to `get_device_count` - 1 |
        | *sn* | Pointer to buffer where the serial number of the device can be stored. 100 or more bytes must be allocated. This is used when connecting to a device |
        +------------+------------+
        
        :param index: Index of device ranges from 0 to `get_device_count` - 1
        :type index: int
        :return: The serial number of a detected device
        :rtype: str
        :raises: :exc:`DeviceReturnError`
        """

        device_sn = create_string_buffer(GET_STRING_SIZE)
        res = self.dgilib.get_device_serial(index, byref(device_sn))
        if self.verbose:
            print(f"\t{res} get_device_serial: {device_sn.value}")
        if res:
            raise DeviceReturnError(f"get_device_serial returned: {res}")
        return device_sn.value

    def is_msd_mode(self, device_sn):
        """`is_msd_mode`
        EDBG devices can be set to a mass storage mode where the DGI is unavailable. In such cases the 
        device is still detected by DGILib, but it won't be possible to directly connect to it. This command is used 
        to check if the device is in such a mode.

        A non-zero return value indicates that the mode must be changed by `set_mode` before proceeding.

        `int is_msd_mode(char* sn)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *sn* | Serial number of the device to check |
        +------------+------------+
        
        :param device_sn: Serial number of the device to check (defaults to self.device_sn)
        :type device_sn: str or None
        :return: A non-zero return value indicates that the mode must be changed by `set_mode` before proceeding.
        :rtype: int
        """

        msd_mode = self.dgilib.is_msd_mode(device_sn)
        if self.verbose:
            print(f"msd_mode: {msd_mode}")
        return msd_mode

    def set_mode(self, device_sn, nmbed=1):
        """`set_mode`
        This function is used to temporarily set the EDBG to a specified mode.

        `int set_mode(char* sn, int nmbed)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *sn* | Serial number of the device to set |
        | *nmbed* | 0 - Set to mbed mode. 1 - Set to DGI mode |
        +------------+------------+
        
        :param device_sn: Serial number of the device to set
        :type device_sn: str
        :param nmbed: 0 - Set to mbed mode. 1 - Set to DGI mode (defaults to DGI mode)
        :type nmbed: int
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.set_mode(device_sn, nmbed)
        if self.verbose:
            print(f"\t{res} set_mode {nmbed}")
        if res:
            raise DeviceReturnError(f"set_mode returned: {res}")

In [5]:
# https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER

"""This module provides Python bindings for the Housekeeping API of DGILib."""

__license__ = "MIT"
__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "1.6.0"
__revision__ = " $Id: dgilib_housekeeping.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"


class DGILibHousekeeping(object):
    """Python bindings for DGILib Housekeeping.
    
    DGILib is a Dynamic-Link Library (DLL) to help software applications communicate with Data Gateway
    Interface (DGI) devices. See the Data Gateway Interface user guide for further details. DGILib handles
    the low-level USB communication and adds a level of buffering for minimizing the chance of overflows.
    The library helps parse data streams of high complexity.
    """

    def __enter__(self):
        """
        :raises: :exc:`DeviceConnectionError`
        """

        self.dgi_hndl = self.connect(self.device_sn)
        c_status = self.connection_status()
        if c_status:
            raise DeviceConnectionError(
                f"Could not connect to device. Connection status: {c_status}."
            )

        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.disconnect()
        if self.verbose:
            print("bye from Housekeeping")

    def connect(self, device_sn):
        """`connect`
        Opens a connection to the specified device. This function must be called prior to any function requiring
        the connection handle.

        `int connect(char* sn, uint32_t* dgi_hndl_p)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *sn* | Buffer holding the serial number of the device to open a connection to |
        | *dgi_hndl_p* | Pointer to a variable that will hold the handle of the connection |
        +------------+------------+
        
        :param device_sn: Serial number of the device
        :type device_sn: str
        :return: Variable that holds the handle of the connection
        :rtype: c_uint()
        :raises: :exc:`DeviceReturnError`
        """

        dgi_hndl = c_uint()  # Create the dgi_hndl

        # Initialize (not in manual, exists in dgilib.h)
        # self.dgilib.Initialize(byref(dgi_hndl))

        res = self.dgilib.connect(device_sn, byref(dgi_hndl))
        if self.verbose:
            print(f"\t{res} connect")
        if res:
            raise DeviceReturnError(f"connect returned: {res}")

        return dgi_hndl

    def disconnect(self):
        """`disconnect`
        Closes the specified connection.

        `int disconnect(uint32_t dgi_hndl)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        +------------+------------+
        
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.disconnect(self.dgi_hndl)
        if self.verbose:
            print(f"\t{res} disconnect")
        if res:
            raise DeviceReturnError(f"disconnect returned: {res}")

        # UnInitialize (not in manual, exists in dgilib.h)
        # self.dgilib.UnInitialize(dgi_hndl)

    def connection_status(self):
        """`connection_status`
        Verifies that the specified connection is still open.

        `int connection_status(uint32_t* dgi_hndl)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        +------------+------------+
        
        :return: A non-zero return value indicates a connection error.
        :rtype: int
        """

        c_status = self.dgilib.connection_status(self.dgi_hndl)
        if self.verbose:
            print(f"connection_status: {c_status}")

        return c_status

    def get_major_version(self):
        """`get_major_version`
        Get the major version of the DGI library.

        `int get_major_version(void)`
        
        :return: The major version of the DGI library
        :rtype: int
        """

        major_version = self.dgilib.get_major_version()

        if self.verbose:
            print(f"major_version: {major_version}")

        return major_version

    def get_minor_version(self):
        """`get_minor_version`
        Get the minor version of the DGI library.

        `int get_minor_version(void)`
        
        :return: The minor version of the DGI library
        :rtype: int
        """

        minor_version = self.dgilib.get_minor_version()

        if self.verbose:
            print(f"minor_version: {minor_version}")

        return minor_version

    def get_build_number(self):
        """`get_build_number`
        Get the major version of the DGI library.
        
        Returns the build number of DGILib. If not supported, returns 0.

        `int get_build_number(void)`
        
        :return: The build number of DGILib. If not supported, returns 0.
        :rtype: int
        """

        build_number = self.dgilib.get_build_number()

        if self.verbose:
            print(f"build_number: {build_number}")

        return build_number

    def get_fw_version(self):
        """`get_fw_version`
        Gets the firmware version of the DGI device connected. Note that this is the version of the DGI device,
        and not the tool.
        
        `int get_fw_version(uint32_t dgi_hndl, unsigned char* major, unsigned char* minor)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *major* | Pointer to a variable where the major version will be stored |
        | *minor* | Pointer to a variable where the minor version will be stored |
        +------------+------------+
        
        :return: Version information of the DGI device:
            - major_fw: the major firmware version of the DGI device connected
            - minor_fw: the minor firmware version of the DGI device connected
        :rtype: tuple(int, int)
        """

        major_fw = c_ubyte()
        minor_fw = c_ubyte()
        res = self.dgilib.get_fw_version(
            self.dgi_hndl, byref(major_fw), byref(minor_fw)
        )
        if self.verbose:
            print(f"\t{res} get_fw_version")
        if res:
            raise DeviceReturnError(f"get_fw_version returned: {res}")
        if self.verbose:
            print(f"major_fw: {major_fw.value}\nminor_fw: {minor_fw.value}")

        return (major_fw.value, minor_fw.value)

    def start_polling(self):
        """`start_polling`
        This function will start the polling system and start acquisition on enabled interfaces. It is possible to
        enable/disable interfaces both before and after the polling has been started. However, no data will be
        transferred until the polling is started.
        
        `int start_polling(uint32_t dgi_hndl)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection
        :type dgi_hndl: c_uint()
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.start_polling(self.dgi_hndl)
        if self.verbose:
            print(f"\t{res} start_polling")
        if res:
            raise DeviceReturnError(f"start_polling returned: {res}")

    def stop_polling(self):
        """`stop_polling`
        This function will stop the polling system and stop acquisition on all interfaces.
        
        `int stop_polling(uint32_t dgi_hndl)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        +------------+------------+
        
        :param dgi_hndl: Handle of the connection
        :type dgi_hndl: c_uint()
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.stop_polling(self.dgi_hndl)
        if self.verbose:
            print(f"\t{res} stop_polling")
        if res:
            raise DeviceReturnError(f"stop_polling returned: {res}")

    def target_reset(self, hold_reset):
        """`target_reset`
        This function is used to control the state of the reset line connected to the target, if available.
        
        `int target_reset(uint32_t dgi_hndl, bool hold_reset)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *hold_reset* | True will assert reset, false will release it |
        +------------+------------+
        
        :param hold_reset: True will assert reset, False will release it
        :type hold_reset: bool
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.target_reset(self.dgi_hndl, hold_reset)
        if self.verbose:
            print(f"\t{res} target_reset")
        if res:
            raise DeviceReturnError(f"target_reset returned: {res}")

In [28]:
# https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER

"""This module provides Python bindings for the Interface Communication API of DGILib."""

__license__ = "MIT"
__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "1.6.0"
__revision__ = (
    " $Id: dgilib_interface_communication.py 1586 2019-02-13 15:56:25Z EWouters $ "
)
__docformat__ = "reStructuredText"


class DGILibInterfaceCommunication(object):
    """Python bindings for DGILib Interface Communication.
    
    DGILib is a Dynamic-Link Library (DLL) to help software applications communicate with Data Gateway
    Interface (DGI) devices. See the Data Gateway Interface user guide for further details. DGILib handles
    the low-level USB communication and adds a level of buffering for minimizing the chance of overflows.
    The library helps parse data streams of high complexity. The timestamp interface is parsed and split into
    separate buffers for each data source.
    """

    def interface_list(self):
        """`interface_list`
        Queries the connected DGI device for available interfaces. Refer to the DGI documentation to resolve the
        ID.
        
        `int interface_list(uint32_t dgi_hndl, unsigned char* interfaces, unsigned char* count)`
        
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *interfaces* | Buffer to hold the ID of the available interfaces. Should be able to hold minimum 10 elements, but a larger count should be used to be future proof. |
        | *count* | Pointer to a variable that will be set to the number of interfaces registered in buffer. |
        +------------+------------+
        
        :return: List of available interfaces
        :rtype: list(int)
        :raises: :exc:`DeviceReturnError`
        """

        interfaces = (c_ubyte * NUM_INTERFACES)()
        interfaceCount = c_ubyte()
        res = self.dgilib.interface_list(
            self.dgi_hndl, byref(interfaces), byref(interfaceCount)
        )
        if self.verbose:
            print(
                f"\t{res} interface_list: {interfaces[:interfaceCount.value]}, interfaceCount: {interfaceCount.value}"
            )
        if res:
            raise DeviceReturnError(f"interface_list returned: {res}")

        return interfaces[: interfaceCount.value]

    def interface_enable(self, interface_id, timestamp=True):
        """`interface_enable`
        Enables the specified interface. Note that no data acquisition will begin until a session has been started.

        `int interface_enable(uint32_t dgi_hndl, int interface_id, bool timestamp)`
     
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *interface_id* | The ID of the interface to enable. |
        | *timestamp* | Setting this to true will make the interface use timestamping. Consult the DGI documentation for details on the timestamping option. |
        +------------+------------+
        
        :param interface_id: The ID of the interface to enable
        :type interface_id: int
        :param timestamp: Setting this to true will make the interface use timestamping (defaults to True)
        :type timestamp: bool
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.interface_enable(self.dgi_hndl, interface_id, timestamp)
        if self.verbose:
            print(f"\t{res} interface_enable: {interface_id}, timestamp: {timestamp}")
        if res:
            raise DeviceReturnError(f"interface_enable: {interface_id}, timestamp: {timestamp} returned: {res}")

    def interface_disable(self, interface_id):
        """`interface_disable`
        Disables the specified interface.

        `int interface_disable(uint32_t dgi_hndl, int interface_id)`
     
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *interface_id* | The ID of the interface to disable. |
        +------------+------------+
        
        :param interface_id: The ID of the interface to disable
        :type interface_id: int
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.interface_disable(self.dgi_hndl, interface_id)
        if self.verbose:
            print(f"\t{res} interface_disable: {interface_id}")
        if res:
            raise DeviceReturnError(f"interface_disable returned: {res}")

    def interface_get_configuration(self, interface_id):
        """`interface_get_configuration`
        Gets the configuration associated with the specified interface. Consult the DGI documentation for details.

        `int interface_get_configuration(uint32_t dgi_hndl, int interface_id, unsigned int* config_id, unsigned int*
        config_value, unsigned int* config_cnt)`
     
        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle to connection |
        | *interface_id* | The ID of the interface |
        | *config_id* | Buffer that will hold the ID field for the configuration item |
        | *config_value* | Buffer that will hold the value field for the configuration item |
        | *config_cnt* | Pointer to variable that will hold the count of stored configuration items |
        +------------+------------+
        
        :param interface_id: The ID of the interface
        :type interface_id: int
        :return: Tuple of a list of configuration IDs and a list of configuration values
        :rtype: tuple(list(int), list(int))
        :raises: :exc:`DeviceReturnError`
        """

        config_id = (c_uint * NUM_CONFIG_IDS)()
        config_value = (c_uint * NUM_CONFIG_IDS)()
        config_cnt = c_uint()
        res = self.dgilib.interface_get_configuration(
            self.dgi_hndl,
            interface_id,
            byref(config_id),
            byref(config_value),
            byref(config_cnt),
        )
        if self.verbose:
            print(
                f"\t{res} interface_get_configuration: {interface_id}, config_cnt: {config_cnt.value}"
            )
        if self.verbose >= 2:
            for i in range(config_cnt.value):
                print(f"\t\tconfig_id: {config_id[i]},\tvalue: {config_value[i]}")
        if res:
            raise DeviceReturnError(f"interface_get_configuration: {interface_id}, timestamp: {timestamp} returned: {res}")

        return config_id[: config_cnt.value], config_value[: config_cnt.value]

    def interface_set_configuration(self, interface_id, config_id, config_value):
        """`interface_set_configuration`
        Sets the given configuration fields for the specified interface. Consult the DGI documentation for details.

        `int interface_set_configuration(uint32_t dgi_hndl, int interface_id, unsigned int* config_id, unsigned int*
        config_value, unsigned int config_cnt)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle to connection |
        | *interface_id* | The ID of the interface |
        | *config_id* | Buffer that holds the ID field for the configuration items to set |
        | *config_value* | Buffer that holds the value field for the configuration items to set |
        | *config_cnt* | Number of items to set |
        +------------+------------+
        
        :param interface_id: The ID of the interface
        :type interface_id: int
        :param config_id: List that holds the ID field for the configuration items to set
        :type config_id: list(int)
        :param config_value: List that holds the value field for the configuration items to set (must have the same
            number of elements as config_id)
        :type config_value: list(int)
        :raises: :exc:`DeviceArgumentError`
        :raises: :exc:`DeviceReturnError`
        """

        if len(config_id) == len(config_value):
            config_cnt = c_uint(len(config_id))
        else:
            raise DeviceArgumentError(
                f"interface_set_configuration: the lenght of config_id list ({len(config_id)}) is not equal to the lenght of config_value list ({len(config_value)})"
            )

        config_id = (c_uint * NUM_CONFIG_IDS)(*config_id)
        config_value = (c_uint * NUM_CONFIG_IDS)(*config_value)
        res = self.dgilib.interface_set_configuration(
            self.dgi_hndl,
            interface_id,
            byref(config_id),
            byref(config_value),
            config_cnt,
        )
        if self.verbose:
            print(
                f"\t{res} interface_set_configuration: {interface_id}, config_cnt: {config_cnt.value}"
            )
        if self.verbose >= 2:
            for i in range(config_cnt.value):
                print(f"\t\tconfig_id: {config_id[i]},\tvalue: {config_value[i]}")
        if res:
            raise DeviceReturnError(f"interface_set_configuration: {interface_id} returned: {res}")

    def interface_clear_buffer(self, interface_id):
        """`interface_clear_buffer`
        Clears the data in the buffers for the specified interface.

        `int interface_clear_buffer(uint32_t dgi_hndl, int interface_id)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *interface_id* | The ID of the interface |
        +------------+------------+
        
        :param interface_id: The ID of the interface
        :type interface_id: int
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.interface_clear_buffer(self.dgi_hndl, interface_id)
        if self.verbose:
            print(f"\t{res} interface_clear_buffer: {interface_id}")
        if res:
            raise DeviceReturnError(f"interface_clear_buffer returned: {res}")

    def interface_read_data(self, interface_id):
        """`interface_read_data`
        Reads the data received on the specified interface. This should be called regularly to avoid overflows in
        the system. DGILib can buffer 10M samples.

        `int interface_read_data(uint32_t dgi_hndl, int interface_id, unsigned char* buffer, unsigned long long*
        timestamp, int* length, unsigned int* ovf_index, unsigned int* ovf_length, unsigned int* ovf_entry_count)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *interface_id* | The ID of the interface |
        | *buffer* | Buffer that will hold the received data. The buffer must have allocated 10M elements. |
        | *timestamp* | If timestamp is enabled for the interface, the buffer that will hold the received data. The buffer must have allocated 10M elements. Otherwise send 0. |
        | *length* | Pointer to a variable that will hold the count of elements received |
        | *ovf_index* | Reserved. Set to 0. |
        | *ovf_length* | Reserved. Set to 0. |
        | *ovf_entry_count* | Reserved. Set to 0. Could be set to a pointer to a variable that can be used as an indicator of overflows. Overflow would be indicated by non-zero value. |
        +------------+------------+
        
        :param interface_id: The ID of the interface
        :type interface_id: int
        :return: Tuple of a list of received values and a list of timestamps
        :rtype: tuple(list(int), list(int))
        :raises: :exc:`DeviceReturnError`
        """

        buffer = (c_ubyte * BUFFER_SIZE)()
        timestamp = (c_ulonglong * BUFFER_SIZE)()
        length = c_uint(0)
        ovf_index = c_uint(0)
        ovf_length = c_uint(0)
        ovf_entry_count = c_uint(0)
        res = self.dgilib.interface_read_data(
            self.dgi_hndl,
            interface_id,
            buffer,
            timestamp,
            byref(length),
            byref(ovf_index),
            byref(ovf_length),
            byref(ovf_entry_count),
        )
        if self.verbose:
            print(
                f"\t{res} interface_read_data: {interface_id}, length: {length.value}"
            )
        if self.verbose >= 2:
            for i in range(length.value):
                print(f"\t{i}:\tbuffer: {buffer[i]},\ttimestamp: {timestamp[i]}")
        if res:
            raise DeviceReturnError(f"interface_read_data: {interface_id} returned: {res}")

        return buffer[: length.value], timestamp[: length.value]

    def interface_write_data(self, interface_id, buffer):
        """`interface_write_data`
        Writes data to the specified interface. A maximum of 255 elements can be written each time. An error
        return code will be given if data hasn't been written yet.

        TODO: A non-zero return value indicates an error. An error will be returned if the interface is still in the process of
        writing data. Wait a while and try again. The function get_connection_status can be used to verify if there
        is an error condition.

        `int interface_write_data(uint32_t dgi_hndl, int interface_id, unsigned char* buffer, int* length)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *dgi_hndl* | Handle of the connection |
        | *interface_id* | The ID of the interface |
        | *buffer* | Buffer that holds the data to write. The buffer must have allocated 10M elements |
        | *length* | Pointer to a variable that will hold the count of elements received |
        +------------+------------+
        
        :param interface_id: The ID of the interface
        :type interface_id: int
        :param buffer: Buffer that holds the data to write (defaults to None)
        :type buffer: int
        :raises: :exc:`DeviceReturnError`
        """

        length = c_uint(len(buffer))
        buffer = (c_ubyte * BUFFER_SIZE)(*buffer)
        res = self.dgilib.interface_write_data(
            self.dgi_hndl, interface_id, byref(buffer), byref(length)
        )
        if self.verbose:
            print(
                f"\t{res} interface_write_data: {interface_id}, length: {length.value}"
            )
        if self.verbose >= 2:
            for i in range(length.value):
                print(f"\t{i}:\tbuffer: {buffer[i]}")
        if res:
            raise DeviceReturnError(
                f"TODOTODOTODO: interface_write_data: {interface_id} returned: {res}"
            )

In [29]:
# https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER

"""This module provides Python bindings for the Auxiliary API of DGILib."""

__license__ = "MIT"
__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "1.6.0"
__revision__ = " $Id: dgilib_auxiliary.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"


class DGILibAuxiliary(object):
    """Python bindings for DGILib Auxiliary.
    
    DGILib is a Dynamic-Link Library (DLL) to help software applications communicate with Data Gateway
    Interface (DGI) devices. See the Data Gateway Interface user guide for further details. DGILib handles
    the low-level USB communication and adds a level of buffering for minimizing the chance of overflows.
    The library helps parse data streams of high complexity. The timestamp interface is parsed and split into
    separate buffers for each data source. The power interface is optionally parsed and calibrated using an
    auxiliary API.
    
    ## Power
    The power interface (as found on some EDBG kits and Power Debugger) uses a protocol stream and
    calibration scheme that can be tricky to get right. The data rates are also relatively high and the
    calibration procedure could cause issues if not handled efficiently. Therefore some auxiliary functions to
    help with this have been made to perform parsing and calibration.
    """

    def __enter__(self):
        """
        """

        self.power_hndl = self.auxiliary_power_initialize()

        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.auxiliary_power_uninitialize()
        if self.verbose:
            print("bye from Auxiliary")

    def auxiliary_power_initialize(self):
        """`auxiliary_power_initialize`
        Initializes the power parser.

        `int auxiliary_power_initialize(uint32_t* power_hndl_p, uint32_t dgi_hndl)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl_p* | Pointer to variable that will hold the handle to the power parser |
        | *dgi_hndl* | Handle of the connection |
        +------------+------------+
        
        :return: Handle of the power parser
        :rtype: c_uint()
        :raises: :exc:`DeviceReturnError`
        """

        power_hndl = c_uint()

        res = self.dgilib.auxiliary_power_initialize(byref(power_hndl), self.dgi_hndl)
        if self.verbose:
            print(f"\t{res} auxiliary_power_initialize")
        if res:
            raise DeviceReturnError(f"auxiliary_power_initialize returned: {res}")

        return power_hndl

    def auxiliary_power_uninitialize(self):
        """`auxiliary_power_uninitialize`
        Uninitializes the power parser.

        `int auxiliary_power_uninitialize(uint32_t power_hndl)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        +------------+------------+
        
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.auxiliary_power_uninitialize(self.power_hndl)
        if self.verbose:
            print(f"\t{res} auxiliary_power_uninitialize")
        if res:
            raise DeviceReturnError(f"auxiliary_power_uninitialize returned: {res}")

    def auxiliary_power_register_buffer_pointers(
        self,
        buffer=None,
        timestamp=None,
        count=None,
        max_count=None,
        channel=0,
        power_type=0,
    ):
        """`auxiliary_power_register_buffer_pointers`
        Registers a set of pointers to be used for storing the calibrated power data. The buffers can then be
        locked by auxiliary_power_lock_data_for_reading, and the data directly read from the specified buffers.
        Zero-pointers can be specified to get the buffers allocated within DGILib. This requires the data to be
        fetched using auxiliary_power_copy_data.

        `int auxiliary_power_register_buffer_pointers(uint32_t power_hndl, float* buffer, double* timestamp, size_t*
        count, size_t max_count, int channel, int type)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        | *buffer* | Buffer that will hold the samples. Set to 0 for automatically allocated. |
        | *timestamp* | Buffer that will hold the timestamp for the samples. Set to 0 for automatically allocated. |
        | *count* | Pointer to a variable that will hold the count of samples. Set to 0 for automatically allocated. |
        | *max_count* | Number of samples that can fit into the specified buffers. Or size of automatically allocated buffers. |
        | *channel* | Power channel for this buffer: A = 0, B = 1 (Power Debugger specific) |
        | *type* | Type of power data: Current = 0, Voltage = 1, Range = 2 |
        +------------+------------+
        
        TODO
        :param channel: Power channel for this buffer: A = 0, B = 1 (defaults to 0)
        :type channel: int
        :param power_type: Type of power data: Current = 0, Voltage = 1, Range = 2 (defaults to 0)
        :type power_type: int
        :raises: :exc:`DeviceReturnError`
        """

#         buffer = (c_float * BUFFER_SIZE)()
#         timestamp = (c_double * BUFFER_SIZE)()
        count = c_size_t()
        max_count = c_size_t
        channel = c_int(channel)
        power_type = c_int(power_type)

        res = self.dgilib.auxiliary_power_register_buffer_pointers(
            self.power_hndl,
            byref(buffer),
            byref(timestamp),
            byref(count),
            max_count,
            channel,
            power_type,
        )
        if self.verbose:
            print(f"\t{res} auxiliary_power_register_buffer_pointers")
        if res:
            raise DeviceReturnError(
                f"auxiliary_power_register_buffer_pointers returned: {res}"
            )

    def auxiliary_power_unregister_buffer_pointers(self, channel=0, power_type=0):
        """`auxiliary_power_unregister_buffer_pointers`
        Unregisters the pointers for the specified power channel.

        `int auxiliary_power_unregister_buffer_pointers(uint32_t power_hndl, int channel, int type)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        | *channel* | Power channel for this buffer: A = 0, B = 1 (Power Debugger specific) |
        | *type* | Type of power data: Current = 0, Voltage = 1, Range = 2 |
        +------------+------------+
        
        :param channel: Power channel for this buffer: A = 0, B = 1 (defaults to 0)
        :type channel: int
        :param power_type: Type of power data: Current = 0, Voltage = 1, Range = 2 (defaults to 0)
        :type power_type: int
        :raises: :exc:`DeviceReturnError`
        """

        # TODO? c_int()

        res = self.dgilib.auxiliary_power_unregister_buffer_pointers(
            self.power_hndl, channel, power_type
        )
        if self.verbose:
            print(f"\t{res} auxiliary_power_unregister_buffer_pointers")
        if res:
            raise DeviceReturnError(
                f"auxiliary_power_unregister_buffer_pointers returned: {res}"
            )

    def auxiliary_power_calibration_is_valid(self):
        """`auxiliary_power_calibration_is_valid`
        Checks the status of the stored calibration.

        Returns true if the calibration is valid, false otherwise. Unity gain and offset will be used.

        `bool auxiliary_power_calibration_is_valid(uint32_t power_hndl)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        +------------+------------+
        
        :return: True if the calibration is valid, False otherwise
        :rtype: bool
        """

        calibration_is_valid = self.dgilib.auxiliary_power_calibration_is_valid(
            self.power_hndl
        )
        if self.verbose:
            print(f"auxiliary_power_calibration_is_valid: {calibration_is_valid}")

        return bool(calibration_is_valid)

    def auxiliary_power_trigger_calibration(self, circuit_type=XAM):
        """`auxiliary_power_trigger_calibration`
        Triggers a calibration of the specified type. This can take some time, so use `auxiliary_power_get_status`
        to check for completion.

        `int auxiliary_power_trigger_calibration(uint32_t power_hndl, int type)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        | *type* | Type of calibration to trigger. See the DGI documentation for details. |
        +------------+------------+
        
        :param circuit_type: Type of calibration to trigger (defaults to XAM)
        :type circuit_type: int
        :raises: :exc:`DeviceReturnError`
        """

        circuit_type = c_int(circuit_type)
        res = self.dgilib.auxiliary_power_trigger_calibration(
            self.power_hndl, circuit_type
        )
        if self.verbose:
            print(f"\t{res} auxiliary_power_trigger_calibration")
        if res:
            raise DeviceReturnError(
                f"auxiliary_power_trigger_calibration returned: {res}"
            )

    def auxiliary_power_get_calibration(self, length=NUM_CALIBRATION):
        """`auxiliary_power_get_calibration`
        Gets the raw calibration read from the tool.

        `int auxiliary_power_get_calibration(uint32_t power_hndl, uint8_t* data, size_t length)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        | *data* | Buffer that will hold the read raw calibration data |
        | *length* | Number of raw calibration bytes to fetch. See the DGI documentation for number of bytes. |
        +------------+------------+
        
        :param length: Number of raw calibration bytes to fetch. See the DGI documentation for number of bytes. (defaults to NUM_CALIBRATION)
        :type length: int
        :return: List of the read raw calibration data
        :rtype: List(int)
        :raises: :exc:`DeviceReturnError`
        """

        data = (c_ubyte * length)()
        length = c_size_t(length)
        res = self.dgilib.auxiliary_power_get_calibration(
            self.power_hndl, byref(data), length
        )
        if self.verbose:
            print(f"\t{res} auxiliary_power_get_calibration: {length.value}")
        if self.verbose >= 2:
            for i in range(length.value):
                print("\t{i}:\t{data[i]}")
        if res:
            raise DeviceReturnError(f"auxiliary_power_get_calibration returned: {res}")

        return data[:]

    def auxiliary_power_get_circuit_type(self):
        """`auxiliary_power_get_circuit_type`
        Gets the type of power circuit.

        `int auxiliary_power_get_circuit_type(uint32_t power_hndl, int* circuit)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        | *circuit* | Pointer to a variable that will hold the circuit type: OLD_XAM = 0x00, XAM = 0x10, PAM = 0x11, UNKNOWN = 0xFF |
        +------------+------------+
        
        :return: The circuit type: OLD_XAM = 0x00, XAM = 0x10, PAM = 0x11, UNKNOWN = 0xFF
        :rtype: int
        :raises: :exc:`DeviceReturnError`
        """

        circuit = c_int()
        res = self.dgilib.auxiliary_power_get_circuit_type(
            self.power_hndl, byref(circuit)
        )
        if self.verbose:
            print(f"\t{res} auxiliary_power_get_circuit_type: {circuit.value}")
        if res:
            raise DeviceReturnError(f"auxiliary_power_get_circuit_type returned: {res}")

        return circuit.value

    def auxiliary_power_get_status(self):
        """`auxiliary_power_get_status`
        Gets the status of the power parser.

        Return codes
        - `IDLE` = 0x00
        - `RUNNING` = 0x01
        - `DONE` = 0x02
        - `CALIBRATING` = 0x03
        - `INIT_FAILED` = 0x10
        - `OVERFLOWED` = 0x11
        - `USB_DISCONNECTED` = 0x12
        - `CALIBRATION_FAILED` = 0x20

        `int auxiliary_power_get_status(uint32_t power_hndl)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        +------------+------------+
        
        :return: The status of the power parser:
            - `IDLE` = 0x00
            - `RUNNING` = 0x01
            - `DONE` = 0x02
            - `CALIBRATING` = 0x03
            - `INIT_FAILED` = 0x10
            - `OVERFLOWED` = 0x11
            - `USB_DISCONNECTED` = 0x12
            - `CALIBRATION_FAILED` = 0x20
        :rtype: int
        """

        status = c_int(dgilib.auxiliary_power_get_status(self.power_hndl))
        if self.verbose:
            print(f"power_status: {status.value}")

        return status.value

    def auxiliary_power_start(self, mode=0, parameter=None):
        """`auxiliary_power_start`
        Starts parsing of power data. The power and power sync interfaces are enabled automatically, but note
        that it is necessary to start the polling separately. This only starts the parser that consumes data from the
        DGILib buffer.

        `int auxiliary_power_start(uint32_t power_hndl, int mode, int parameter)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        | *mode* | Sets the mode of capture. |
        |  | 0 - continuous capturing which requires the user to periodically consume the data. |
        |  | 1 - oneshot capturing that captures data until the buffer has been read once, has been filled or the time from the first received sample in seconds equals the specified parameter. |
        | *parameter* | Mode specific |
        +------------+------------+
        
        :param mode: Sets the mode of capture (defaults to 0)
            - 0: continuous capturing which requires the user to periodically consume the data
            - 1: oneshot capturing that captures data until the buffer has been read once, has been 
            filled or the time from the first received sample in seconds equals the specified parameter
        :type mode: int
        :param parameter: Mode specific (defaults to None)
        :type parameter: int or None
        :raises: :exc:`DeviceReturnError`
        """

        mode = c_int(mode)
        parameter = c_int(parameter)
        res = self.dgilib.auxiliary_power_start(self.power_hndl, mode, parameter)
        if self.verbose:
            print(
                f"\t{res} auxiliary_power_start, mode: {mode}, parameter: {parameter}"
            )
        if res:
            raise DeviceReturnError(f"auxiliary_power_start returned: {res}")

    def auxiliary_power_stop(self):
        """`auxiliary_power_stop`
        Stops parsing of power data.

        `int auxiliary_power_stop(uint32_t power_hndl)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        +------------+------------+
        
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.auxiliary_power_stop(self.power_hndl)
        if self.verbose:
            print(f"\t{res} auxiliary_power_stop, mode: {mode}, parameter: {parameter}")
        if res:
            raise DeviceReturnError(f"auxiliary_power_stop returned: {res}")

    def auxiliary_power_lock_data_for_reading(self):
        """`auxiliary_power_lock_data_for_reading`
        Blocks the parsing thread from accessing all the buffers. This must be called before the user application
        code accesses the buffers, or a call to `auxiliary_power_copy_data` is made. Afterwards
        `auxiliary_power_free_data` must be called. Minimize the amount of time between locking and freeing to
        avoid buffer overflows.

        `int auxiliary_power_lock_data_for_reading(uint32_t power_hndl)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        +------------+------------+
        
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.auxiliary_power_lock_data_for_reading(self.power_hndl)
        if self.verbose:
            print(f"\t{res} auxiliary_power_lock_data_for_reading")
        if res:
            raise DeviceReturnError(
                f"auxiliary_power_lock_data_for_reading returned: {res}"
            )

    def auxiliary_power_copy_data(self, max_count=BUFFER_SIZE, channel=0, power_type=0):
        """`auxiliary_power_copy_data`
        Copies parsed power data into the specified buffer. Remember to lock the buffers first. If the count
        parameter is the same as max_count there is probably more data to be read. Do another read to get the
        remaining data.

        `int auxiliary_power_copy_data(uint32_t power_hndl, float* buffer, double* timestamp, size_t* count, size_t
        max_count, int channel, int type)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        | *buffer* | Buffer that will hold the samples. |
        | *timestamp* | Buffer that will hold the timestamp for the samples. |
        | *count* | Pointer to a variable that will hold the count of elements copied |
        | *max_count* | Maximum number of elements that the buffer can hold |
        | *channel* | Power channel for this buffer: A = 0, B = 1 (Power Debugger specific) |
        | *type* | Type of power data: Current = 0, Voltage = 1, Range = 2 |
        +------------+------------+
        
        :param max_count: Maximum number of elements that the buffer can hold (defaults to BUFFER_SIZE)
        :type max_count: int
        :param channel: Power channel for this buffer: A = 0, B = 1 (defaults to 0)
        :type channel: int
        :param power_type: Type of power data: Current = 0, Voltage = 1, Range = 2 (defaults to 0)
        :type power_type: int
        :return: Tuple of a list of samples and a list of the timestamps for the samples
        :rtype: tuple(list(int), list(int))
        :raises: :exc:`DeviceReturnError`
        """

        buffer = (c_float * max_count)()
        timestamp = (c_double * max_count)()
        count = c_size_t()
        max_count = c_size_t(max_count)
        channel = c_int(channel)
        power_type = c_int(power_type)

        res = self.dgilib.auxiliary_power_copy_data(
            self.power_hndl,
            powerBuffer,
            powerTimestamp,
            byref(count),
            max_count,
            channel,
            power_type,
        )
        if self.verbose:
            print(
                f"\t{res} auxiliary_power_copy_data: {count} samples, power_type: {power_type}"
            )
        if res:
            raise DeviceReturnError(f"auxiliary_power_copy_data returned: {res}")

        return buffer[:count], timestamp[:count]

    def auxiliary_power_free_data(self):
        """`auxiliary_power_free_data`
        Clears the power data buffers and allows the power parser to continue.

        `int auxiliary_power_free_data(uint32_t power_hndl)`

        +------------+------------+
        | Parameter  | Description |
        +============+============+
        | *power_hndl* | Handle of the power parser |
        +------------+------------+
        
        :raises: :exc:`DeviceReturnError`
        """

        res = self.dgilib.auxiliary_power_free_data(self.power_hndl)
        if self.verbose:
            print(f"\t{res} auxiliary_power_free_data")
        if res:
            raise DeviceReturnError(f"auxiliary_power_free_data returned: {res}")

In [30]:
# https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER

"""This module provides Python bindings for DGILib."""

__license__ = "MIT"
__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "1.6.0"
__revision__ = " $Id: dgilib.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"

# from dgilib_discovery import DGILibDiscovery
# from dgilib_housekeeping import DGILibHousekeeping
# from dgilib_interface_communication import DGILibInterfaceCommunication
# from dgilib_auxiliary import DGILibAuxiliary


class DGILib(
    DGILibDiscovery, DGILibHousekeeping, DGILibInterfaceCommunication, DGILibAuxiliary
):
    """Python bindings for DGILib.
    
    DGILib is a Dynamic-Link Library (DLL) to help software applications communicate with Data Gateway
    Interface (DGI) devices. See the Data Gateway Interface user guide for further details. DGILib handles
    the low-level USB communication and adds a level of buffering for minimizing the chance of overflows.
    The library helps parse data streams of high complexity. The timestamp interface is parsed and split into
    separate buffers for each data source. The power interface is optionally parsed and calibrated using an
    auxiliary API.
    
    :Example:

    >>> with DGILib() as dgilib:
    ...     dgilib.info(True)
    major_version: 5, minor_version: 7, build_number: 244
    """

    def __init__(
        self, dgilib_path="dgilib", device_index=None, device_sn=None, verbose=1
    ):
        """
        :param dgilib_path: Path to dgilib.dll (More info at: https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER)
        :type dgilib_path: str
        :param device_index: index of the device to use, only usefull if multiple devices are connected
            (default is None, will resolve to 0)
        :type device_index: int or None
        :param device_sn: the serial number of the device to use. Has higher priority than device_index
            (default is None, will resolve to serial number of device 0)
        :type device_sn: str or None
        :param verbose: set to a positive number to print more status messages (default is 0)
        :type verbose: int
        :raises: :exc:`DLLError`
        """

        # Load the dgilib.dll
        try:
            self.dgilib = cdll.LoadLibrary(dgilib_path)
        except OSError as e:
            raise DLLError(
                f"dgilib.dll could not be found in the specicied path: {dgilib_path}."
            )

#         DGILibDiscovery.__init__(self)
#         DGILibHousekeeping.__init__(self)
#         DGILibInterfaceCommunication.__init__(self)
#         DGILibAuxiliary.__init__(self)

        self.device_index = device_index
        self.device_sn = device_sn
        self.verbose = verbose

    def __enter__(self):
        """
        :raises: :exc:`DeviceIndexError`
        :raises: :exc:`DeviceConnectionError`
        """

        DGILibDiscovery.__enter__(self)
        DGILibHousekeeping.__enter__(self)
#         DGILibInterfaceCommunication.__enter__(self)
        DGILibAuxiliary.__enter__(self)

#         self.connect()
#         c_status = self.connection_status()
#         if c_status:
#             raise DeviceConnectionError(
#                 f"Could not connect to device. Connection status: {c_status}."
#             )
#         self.interface_list()
#         self.interface_enable(None, INTERFACE_GPIO, True)

#         self.start_polling()

        return self

    def __exit__(self, exc_type, exc_value, traceback):

        DGILibAuxiliary.__exit__(self, exc_type, exc_value, traceback)
#         DGILibInterfaceCommunication.__exit__(self, exc_type, exc_value, traceback)
        DGILibHousekeeping.__exit__(self, exc_type, exc_value, traceback)
        DGILibDiscovery.__exit__(self, exc_type, exc_value, traceback)
#         self.stop_polling()
#         self.interface_disable(None, INTERFACE_GPIO)
#         self.disconnect()
        if self.verbose:
            print("bye from DGILib")

In [31]:
# https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER

"""This module provides Python bindings for DGILib Extra."""

__license__ = "MIT"
__author__ = "Erik Wouters <ehwo(at)kth.se>"
__credits__ = "Atmel Corporation. / Rev.: Atmel-42771A-DGILib_User Guide-09/2016"
__license__ = "MIT"
__version__ = "1.6.0"
__revision__ = " $Id: dgilib_extra.py 1586 2019-02-13 15:56:25Z EWouters $ "
__docformat__ = "reStructuredText"

# from dgilib import DGILib


class DGILibExtra(DGILib):
    """Python bindings for DGILib Extra.
    """

    def __init__(
        self, dgilib_path="dgilib", device_index=None, device_sn=None, verbose=1
    ):
        """
        :param dgilib_path: Path to dgilib.dll (More info at: https://www.microchip.com/developmenttools/ProductDetails/ATPOWERDEBUGGER)
        :type dgilib_path: str
        :param device_index: index of the device to use, only usefull if multiple devices are connected
            (default is None, will resolve to 0)
        :type device_index: int or None
        :param device_sn: the serial number of the device to use. Has higher priority than device_index
            (default is None, will resolve to serial number of device 0)
        :type device_sn: str or None
        :param verbose: set to a positive number to print more status messages (default is 0)
        :type verbose: int
        :raises: :exc:`DLLError`
        """

        DGILib.__init__(self, dgilib_path, device_index, device_sn, verbose)

    def __enter__(self):
        """
        :raises: :exc:`DeviceIndexError`
        :raises: :exc:`DeviceConnectionError`
        """

        DGILib.__enter__(self)

#         self.connect()
#         c_status = self.connection_status()
#         if c_status:
#             raise DeviceConnectionError(
#                 f"Could not connect to device. Connection status: {c_status}."
#             )
#         self.interface_list()
#         self.interface_enable(None, INTERFACE_GPIO, True)

#         self.start_polling()

        return self

    def __exit__(self, exc_type, exc_value, traceback):

#         self.stop_polling()
#         self.interface_disable(None, INTERFACE_GPIO)
#         self.disconnect()

        DGILib.__exit__(self, exc_type, exc_value, traceback)
        if self.verbose:
            print("bye from DGILib Extra")

    def info(self):
        """Get the build information of DGILib.
        
        :param print_info: A flag used to print the build information to the console (default is False)
        :type print_info: bool
        :return:  Version information of DGILib:
            - major_version: the major_version of DGILib
            - minor_version: the minor_version of DGILib
            - build_number: the build number of DGILib. 0 if not supported
            - major_fw: the major firmware version of the DGI device connected
            - minor_fw: the minor firmware version of the DGI device connected
        :rtype: tuple
        """

        major_version = self.get_major_version()
        minor_version = self.get_minor_version()
        build_number = self.get_build_number()
        major_fw, minor_fw = self.get_fw_version()

        return (major_version, minor_version, build_number, major_fw, minor_fw)

    def device_reset(self, duration=1):
        """Set the device reset line for duration seconds.
        """

        res = dgilib.target_reset(True)
        sleep(duration)
        res = dgilib.target_reset(False)

In [38]:
with DGILibExtra(verbose=1) as dgilib:
    print(dgilib.info())
    print(dgilib.get_device_name(0))
    print(dgilib.get_fw_version())
    print(dgilib.is_msd_mode(dgilib.device_sn))
    #     print(dgilib.set_mode( dgilib.device_sn, 0))
    #     print(dgilib.is_msd_mode(dgilib.device_sn))
    #     print(dgilib.set_mode(dgilib.device_sn, 1))

    print(dgilib.connection_status())
    print(dgilib.start_polling())
    print(dgilib.stop_polling())
    print(dgilib.device_reset())

    interface_list = dgilib.interface_list()
    for interface_id in (INTERFACE_SPI, INTERFACE_USART, INTERFACE_I2C, INTERFACE_GPIO):
        if interface_id in interface_list:
            print(dgilib.interface_enable(interface_id))
    dgilib.start_polling()

    for interface_id in interface_list:
        print(dgilib.interface_get_configuration(interface_id))
    for interface_id in (
        INTERFACE_TIMESTAMP,
        INTERFACE_SPI,
        INTERFACE_USART,
        INTERFACE_I2C,
        INTERFACE_GPIO,
        INTERFACE_POWER_SYNC,
        INTERFACE_RESERVED,
    ):
        if interface_id in interface_list:
            print(
                dgilib.interface_set_configuration(
                    interface_id, *dgilib.interface_get_configuration(interface_id)
                )
            )

    for interface_id in (INTERFACE_SPI, INTERFACE_USART, INTERFACE_I2C, INTERFACE_GPIO, INTERFACE_POWER_DATA, INTERFACE_POWER_SYNC, INTERFACE_RESERVED):
        if interface_id in interface_list:
            print(dgilib.interface_read_data(interface_id))
            
    # TODO: test write data
            
            
    dgilib.stop_polling()
    for interface_id in interface_list:
        print(dgilib.interface_clear_buffer(interface_id))

    for interface_id in (INTERFACE_SPI, INTERFACE_USART, INTERFACE_I2C, INTERFACE_GPIO):
        if interface_id in interface_list:
            print(dgilib.interface_disable(interface_id))
            
            
            
    

    sleep(1)

device_count: 1
	0 get_device_serial: b'ATML3138061800001604'
	0 connect
connection_status: 0
	0 auxiliary_power_initialize
major_version: 5
minor_version: 7
build_number: 244
	0 get_fw_version
major_fw: 3
minor_fw: 2
(5, 7, 244, 3, 2)
	0 get_device_name: b'SAM L11 Xplained Pro'
b'SAM L11 Xplained Pro'
	0 get_fw_version
major_fw: 3
minor_fw: 2
(3, 2)
msd_mode: 0
0
connection_status: 0
0
	0 start_polling
None
	0 stop_polling
None
	0 target_reset
	0 target_reset
None
	0 interface_list: [0, 32, 34, 48, 64, 65, 80], interfaceCount: 7
	0 interface_enable: 32, timestamp: True
None
	0 interface_enable: 34, timestamp: True
None
	0 interface_enable: 48, timestamp: True
None
	0 start_polling
	0 interface_get_configuration: 0, config_cnt: 2
([0, 1], [32, 60000000])
	0 interface_get_configuration: 32, config_cnt: 3
([0, 1, 2], [8, 0, 0])
	0 interface_get_configuration: 34, config_cnt: 2
([0, 1], [100000, 40])
	0 interface_get_configuration: 48, config_cnt: 2
([0, 1], [0, 0])
	0 interface_get_confi

In [ ]:
print("Resetting {0}".format(repr(name.value)))
print(dgilib.target_reset(dgi_hndl, True))
time.sleep(1)
print(dgilib.target_reset(dgi_hndl, False))
print("Done");

In [21]:
timerPrescaler = config_value[0]
timerFrequency = config_value[1]
timerFactor = timerPrescaler / timerFrequency
print("timerFactor: {0}".format(timerFactor))

timerFactor: 5.333333333333333e-07


In [24]:
config_value[0] = 0b1111
config_value[1] = 0b0000
res = dgilib.interface_set_configuration(dgi_hndl, interface_id, byref(config_id), byref(config_value), config_cnt)
print("{0} interface_set_configuration: {1}, config_cnt: {2}".format(res, interface_id, repr(config_cnt.value)))
for i in range(config_cnt.value):
    print("\tconfig_id: {0},\tvalue: {1}".format(config_id[i], config_value[i]))

0 interface_set_configuration: 48, config_cnt: 2
	config_id: 0,	value: 15
	config_id: 1,	value: 0


In [33]:
calibrationType = circuit
res = dgilib.auxiliary_power_trigger_calibration(power_hndl, calibrationType)
print("{0} auxiliary_power_trigger_calibration".format(res))
if (not res):
    print("Calibrating {0}".format(name.value))
    while (dgilib.auxiliary_power_get_status(power_hndl) == 0x03):
        print("Calibrating...")
        time.sleep(0.1)
    print("Done")

0 auxiliary_power_trigger_calibration
Calibrating b'SAM L11 Xplained Pro'
Calibrating...
Calibrating...
Done


In [40]:
#     int loggedSamples = 0;
#     printf("Waiting to log some data \n");
#     for (int i = 0; i < 25; i++) {
#         auxiliary_power_lock_data_for_reading(power_hndl);
#         auxiliary_power_copy_data(power_hndl, powerBuffer+loggedSamples, powerTimestamp+loggedSamples, &powerCount, max_count, channel, powerType);
#         auxiliary_power_free_data(power_hndl);
#         loggedSamples += powerCount;
#         printf(".%d", powerCount);
# //        printf(".");
#         for (int j = 0;j < WAIT_ITERATIONS; j+=10) {}
#     }
#     printf(" Done\n");

In [ ]:
print("captured power data:")
for i in range(powerCount.value):
    print("\t{0}: buffer: {1}, timestamp: {2}".format(i, powerBuffer[i], powerTimestamp[i]))

In [49]:
#     interface_id = INTERFACE_GPIO;
#     res = interface_read_data(dgi_hndl, interface_id, readBuffer, timestamp, &length, &ovf_index, &ovf_length, &ovf_entry_count);
#     printf("%d interface_read_data: 0x%.2x, length: %d\n", res, interface_id, length);
#     for (int i = 0; i < length; ++i) {
#         printf("\t%.6d: buffer: %.4u, timestamp: %.4llu, time: %lf\n", i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor);
#     }

In [50]:
#     res = interface_read_data(dgi_hndl, INTERFACE_POWER_SYNC, readBuffer, timestamp, &length, &ovf_index, &ovf_length, &ovf_entry_count);
#     printf("%d interface_read_data: 0x%.2x, length: %d\n", res, INTERFACE_POWER_SYNC, length);
#     for (int i = 0; i < length; ++i) {
#         printf("\t%.6d: buffer: %.4u, timestamp: %.4llu, time: %lf\n", i, readBuffer[i], timestamp[i], timestamp[i] * timerFactor);
#     }